In [1]:
######## Create, Train, and Predict Models

# Loads train, test, and val datasets
# Instantiates Models
# Trains Models
# Predicts using Models

Import All Required Modules

In [11]:
import sys

# Manually add the project root to sys.path
sys.path.append('/Users/joaquinuriarte/Documents/GitHub/sports-betting/')

# === STEP 0: Imports
from modules.model_manager.trainer.trainer import Trainer
from modules.model_manager.predictor.predictor import Predictor
from modules.model_manager.factories.model_factory import ModelFactory
from modules.model_manager.helpers.configuration_loader import ConfigurationLoader as MMConfigLoader
from modules.model_manager.model_manager import ModelManager
from sklearn.model_selection import KFold
from modules.data_structures.model_dataset import ModelDataset
from model_binaries.utils.binary_utils import save_entity, load_entity, cross_val_train, compute_f1, get_subset

In [12]:
# === STEP 1: Dependency Instantiations And Global Variable Declarations

## === MODEL MANAGER
yaml_path0 = '/Users/joaquinuriarte/Documents/GitHub/sports-betting/configs/model_v10/model_v10_000.yaml'
yaml_paths = [yaml_path0]


checkpoint = '/Users/joaquinuriarte/Documents/GitHub/sports-betting/models'
trainer = Trainer(checkpoint)
predictor = Predictor()
model_factory = ModelFactory()
mm_configuration_loader = MMConfigLoader()

Load Train into Memory

In [3]:
# File path to load train, test, and val datasets
train_test_val_folder_path = "/Users/joaquinuriarte/Documents/GitHub/sports-betting/processed_datasets/model_v10"

In [4]:
train_dataset = load_entity(train_test_val_folder_path, "scaled_train.pkl")
val_dataset = load_entity(train_test_val_folder_path, "scaled_val.pkl")
test_dataset = load_entity(train_test_val_folder_path, "scaled_test.pkl")

Create and Train Models

In [5]:
reduced_train_set = get_subset(train_dataset, 500)

In [6]:
# === STEP 4: MODEL MANAGER
yamls = [yaml_path0]
model_manager = ModelManager(trainer, predictor, model_factory, mm_configuration_loader)

In [10]:
models = model_manager.create_models(yamls)
model_manager.train(models, [(reduced_train_set, val_dataset)], save_after_training=True)

INFO:root:Training model '5105f8d3f5191ed990dc61ad415280a1' for 50 epochs with batch size 32.


Epoch 1/50
16/16 [==============================] - 3s 45ms/step - loss: 0.8645 - accuracy: 0.4640 - precision: 0.6343 - recall: 0.3524 - auc: 0.5125 - val_loss: 0.7084 - val_accuracy: 0.5373 - val_precision: 0.5638 - val_recall: 0.6922 - val_auc: 0.5285 - lr: 0.0010
Epoch 2/50
16/16 [==============================] - 0s 22ms/step - loss: 0.4237 - accuracy: 0.7460 - precision: 0.9476 - recall: 0.6317 - auc: 0.8546 - val_loss: 0.7026 - val_accuracy: 0.5118 - val_precision: 0.5613 - val_recall: 0.5047 - val_auc: 0.5197 - lr: 0.0010
Epoch 3/50
16/16 [==============================] - 0s 19ms/step - loss: 0.2878 - accuracy: 0.8740 - precision: 0.9922 - recall: 0.8063 - auc: 0.9625 - val_loss: 0.6984 - val_accuracy: 0.4705 - val_precision: 0.5387 - val_recall: 0.2441 - val_auc: 0.5114 - lr: 0.0010
Epoch 4/50
16/16 [==============================] - 0s 28ms/step - loss: 0.2056 - accuracy: 0.9460 - precision: 1.0000 - recall: 0.9143 - auc: 0.9802 - val_loss: 0.6999 - val_accuracy: 0.4733 - va

INFO:root:Model '5105f8d3f5191ed990dc61ad415280a1': Finished training.


Model saved successfully at: models/5105f8d3f5191ed990dc61ad415280a1/model_weights_5105f8d3f5191ed990dc61ad415280a1.pth


Assess

Launch Tensorboard

In [10]:
%tensorboard --logdir=./logs

UsageError: Line magic function `%tensorboard` not found.
